In [1]:
from pyspark.ml.recommendation import ALS

In [2]:
#create session
appName = "Recommendation System"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
ratings = spark.read.csv('dataset/ratings.csv', inferSchema=True, header=True)
movies = spark.read.csv('dataset/movies.csv', inferSchema=True, header=True)
#merge "movies" and "ratings" dataFrame based on "movieId"
ratings.join(movies, "movieId").show(5)

+-------+------+------+----------+--------------------+--------------------+
|movieId|userId|rating| timestamp|               title|              genres|
+-------+------+------+----------+--------------------+--------------------+
|     31|     1|   2.5|1260759144|Dangerous Minds (...|               Drama|
|   1029|     1|   3.0|1260759179|        Dumbo (1941)|Animation|Childre...|
|   1061|     1|   3.0|1260759182|     Sleepers (1996)|            Thriller|
|   1129|     1|   2.0|1260759185|Escape from New Y...|Action|Adventure|...|
|   1172|     1|   4.0|1260759205|Cinema Paradiso (...|               Drama|
+-------+------+------+----------+--------------------+--------------------+
only showing top 5 rows



In [4]:
#use only column data of "userId", "movieId", dan "rating"
data = ratings.select("userId", "movieId", "rating")
#divide data, 70% for training and 30% for testing
splits = data.randomSplit([0.7, 0.3])
train = splits[0].withColumnRenamed("rating", "label")
test = splits[1].withColumnRenamed("rating", "trueLabel")
#calculate number of rows
train_rows = train.count()
test_rows = test.count()
print ("number of training data rows:", train_rows, 
       ", number of testing data rows:", test_rows)

number of training data rows: 69693 , number of testing data rows: 30311


In [5]:
#define ALS (Alternating Least Square) as our recommender system
als = ALS(maxIter=19, regParam=0.01, userCol="userId", 
          itemCol="movieId", ratingCol="label")
#train our ALS model
model = als.fit(train)
print("Training is done!")

Training is done!


In [6]:
prediction = model.transform(test)
print("testing is done!")

testing is done!


In [7]:
prediction.join(movies, "movieId").select(
    "userId", "title", "prediction", "trueLabel").show(n=10, truncate=False)

+------+--------------------------------+----------+---------+
|userId|title                           |prediction|trueLabel|
+------+--------------------------------+----------+---------+
|575   |Awfully Big Adventure, An (1995)|NaN       |4.0      |
|311   |Guilty as Sin (1993)            |3.464463  |3.0      |
|126   |Hudsucker Proxy, The (1994)     |5.001766  |5.0      |
|460   |Hudsucker Proxy, The (1994)     |3.3995361 |5.0      |
|548   |Hudsucker Proxy, The (1994)     |3.9493232 |4.0      |
|602   |Hudsucker Proxy, The (1994)     |4.1632233 |3.0      |
|452   |Hudsucker Proxy, The (1994)     |3.9542098 |3.0      |
|299   |Hudsucker Proxy, The (1994)     |3.9810295 |4.5      |
|358   |Hudsucker Proxy, The (1994)     |4.538314  |5.0      |
|502   |Hudsucker Proxy, The (1994)     |4.8428755 |4.0      |
+------+--------------------------------+----------+---------+
only showing top 10 rows



In [8]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): nan


In [9]:

prediction.count()
a = prediction.count()
print("number of original data rows: ", a)
#drop rows with any missing data
cleanPred = prediction.dropna(how="any", subset=["prediction"])
b = cleanPred.count()
print("number of rows after dropping data with missing value: ", b)
print("number of missing data: ", a-b)

number of original data rows:  30311
number of rows after dropping data with missing value:  29084
number of missing data:  1227


In [10]:
rmse = evaluator.evaluate(cleanPred)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 1.2495504081379096
